In [1]:
%matplotlib inline
import config
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import psycopg2
from psycopg2 import sql
from IPython.display import display
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pickle

In [2]:
conn = psycopg2.connect(user = config.user,
                              password = config.password,
                              host = config.hostname,
                              port = config.port,
                              database = config.database)

cur = conn.cursor()

# Print PostgreSQL version
cur.execute("SELECT * from profiles;")
record = cur.fetchall()

In [3]:
cur.execute("Select * FROM profiles LIMIT 0")
colnames = [desc[0] for desc in cur.description]
print(colnames)
p = ['username', 'agerange', 'gender', 'orientation', 'currentstatus', 'togethertime', 'relationshipdescription', 'selfeducation', 'selffinancial', 'selfconfidence', 'selfreligious', 'selfmaterialism', 'selfimage', 'selfoccupation', 'selfworkethic', 'selfhousehold', 'selfcommunication', 'selfartsy', 'selfcharitable', 'selfpurpose', 'selfstatus', 'selfcultured', 'selfselfcare', 'selfhonesty', 'selffamily']
print(len(p))

['username', 'agerange', 'gender', 'orientation', 'currentstatus', 'togethertime', 'relationshipdescription', 'selfeducation', 'selffinancial', 'selfconfidence', 'selfreligious', 'selfmaterialism', 'selfimage', 'selfoccupation', 'selfworkethic', 'selfhousehold', 'selfcommunication', 'selfartsy', 'selfcharitable', 'selfpurpose', 'selfstatus', 'selfcultured', 'selfselfcare', 'selfhonesty', 'selffamily', 'value1', 'selfvalue1', 'partnervalue1', 'value2', 'selfvalue2', 'partnervalue2', 'value3', 'selfvalue3', 'partnervalue3', 'value4', 'selfvalue4', 'partnervalue4', 'value5', 'selfvalue5', 'partnervalue5', 'selfappearance', 'selfsocial', 'selfshy', 'selfalpha', 'selfhumorous', 'selfspontaneous', 'selfgenerous', 'selfdriven', 'selfintuitive', 'selfsexual', 'selfopenminded', 'selfvibe', 'selfovergiving', 'selfdominant', 'type1', 'partnertype1', 'type2', 'partnertype2', 'type3', 'partnertype3', 'type4', 'partnertype4', 'type5', 'partnertype5', 's_decisionmakingprocess', 'qp_emotionalintellige

In [8]:

lovedata = pd.DataFrame(record)

lovedata = lovedata.dropna(subset=lovedata.columns[25:39])

In [5]:
# update list if there are any fields added to data
features = pd.DataFrame(columns=['Education/Knowledge/Street Smarts',
          'partner_education_score',
          'difference_education_score',
          'Financial Choices',
          'partner_financial',
          'difference_financial',
          'Confidence/Self-Esteem',
          'partner_confidence',
          'difference_confidence',
          'Religious/Spiritual Values',
          'partner_religious',
          'difference_religious',
          'Materialism/Superficiality',
          'partner_materialism',
          'difference_materialism',
          'Image/Fashion Sense/Body Modification',
          'partner_image',
          'differnece_image',
          'Occupation/Work Ethic/Self-Discipline',
          'partner_occupation',
          'difference_occupation',
          'Household Care, Maintenance and Cleanliness',
          'partner_house','difference_house',
          'Communication Style/Manners',
          'partner_communication',
          'difference_communication',
          'Artsy/Creative/Musical',
          'partner_artsy',
          'difference_artsy',
          'Charitable/Philanthropic',
          'partner_charity',
          'difference_charity',
          'Pursuing a Greater Purpose',
          'partner_purpose',
          'difference_purpose',
          'Social Status/Sociability',
          'partner_social',
          'difference_social',
          'Cultured/Well-traveled/"Woke"',
          'partner_culture',
          'difference_culture',
          'Self-Care/Personal Hygiene/Cleanliness',
          'partner_care','difference_care',
          'Honesty/Dependable/Reliable',
          'partner_honesty','difference_honesty','Family Values','partner_family','difference_family'])
features

,Education/Knowledge/Street Smarts,partner_education_score,difference_education_score,Financial Choices,partner_financial,difference_financial,Confidence/Self-Esteem,partner_confidence,difference_confidence,Religious/Spiritual Values,...,difference_culture,Self-Care/Personal Hygiene/Cleanliness,partner_care,difference_care,Honesty/Dependable/Reliable,partner_honesty,difference_honesty,Family Values,partner_family,difference_family


In [6]:
# iterate thru dataframe and use columns from 
for i, j in lovedata.iterrows():
    if i == 0:
        continue
        
    ls = [0 for x in range(len(features.columns))]
        
    for k in range(25,39,3):
        if j[k] in features.columns:
            ls[features.columns.get_loc(j[k])] = int(j[k+1])
            ls[features.columns.get_loc(j[k]) + 1] = int(j[k+2])
            ls[features.columns.get_loc(j[k]) + 2] = abs(int(j[k+1]) - (int(j[k+2])*.99))
        
#     features.loc[i-1]=ls
    features.loc[i]=ls

features

,Education/Knowledge/Street Smarts,partner_education_score,difference_education_score,Financial Choices,partner_financial,difference_financial,Confidence/Self-Esteem,partner_confidence,difference_confidence,Religious/Spiritual Values,...,difference_culture,Self-Care/Personal Hygiene/Cleanliness,partner_care,difference_care,Honesty/Dependable/Reliable,partner_honesty,difference_honesty,Family Values,partner_family,difference_family
1,5.0,6.0,0.94,0.0,0.0,0.00,5.0,4.0,1.04,0.0,...,0.0,0.0,0.0,0.00,6.0,8.0,1.92,4.0,7.0,2.93
2,8.0,8.0,0.08,9.0,10.0,0.90,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,9.0,10.0,0.90,10.0,10.0,0.10
3,8.0,5.0,3.05,6.0,5.0,1.05,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00
4,8.0,8.0,0.08,8.0,8.0,0.08,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,9.0,9.0,0.09,8.0,9.0,0.91
5,7.0,8.0,0.92,0.0,0.0,0.00,0.0,0.0,0.00,0.0,...,0.0,8.0,8.0,0.08,0.0,0.0,0.00,6.0,7.0,0.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,9.0,7.0,2.07,0.0,0.0,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,9.0,5.0,4.05,0.0,0.0,0.00
77,0.0,0.0,0.00,0.0,0.0,0.00,5.0,3.0,2.03,0.0,...,0.0,0.0,0.0,0.00,7.0,5.0,2.05,0.0,0.0,0.00
78,0.0,0.0,0.00,10.0,10.0,0.10,0.0,0.0,0.00,0.0,...,0.0,10.0,10.0,0.10,9.0,10.0,0.90,0.0,0.0,0.00
79,0.0,0.0,0.00,9.0,9.0,0.09,9.0,9.0,0.09,6.0,...,0.0,0.0,0.0,0.00,9.0,9.0,0.09,0.0,0.0,0.00


In [9]:
successful = lovedata.iloc[:,65:95]
successful = successful.fillna(0)
successful.reset_index(inplace=True, drop = True)
successful
total_list = []

for i, column in successful.iterrows():
    total = 0
    for j in range(len(column)):
        if column[j+65] != 0:
            successful.iloc[i,j] = 1
            total += 1
            
    total_list.append(total*10) #multiplied by ten to increase scale take out for true values
print(len(total_list)) 
print(successful.shape)
successful['Total'] = total_list

unsuccess_value = successful.loc[1:,['Total']]

#The Y values
unsuccess_value

69
(69, 30)


,Total
1,40
2,70
3,30
4,50
5,80
...,...
64,60
65,130
66,40
67,20


In [37]:
conn = psycopg2.connect(user = config.user,
                              password = config.password,
                              host = config.hostname,
                              port = config.port,
                              database = config.database)

cur = conn.cursor()

# Print PostgreSQL version
cur.execute("SELECT * from profiles;")
record = cur.fetchall()

feat = pd.DataFrame(columns = ['Education/Knowledge/Street Smarts',
                                 'Financial Choices',
                                 'Confidence/Self-Esteem',
                                 'Religious/Spiritual Values',
                                 'Materialism/Superficiality',
                                 'Image/Fashion Sense/Body Modification',
                                 'Occupation/Work Ethic/Self-Discipline',
                                 'Household Care, Maintenance and Cleanliness',
                                 'Communication Style/Manners',
                                 'Artsy/Creative/Musical',
                                 'Charitable/Philanthropic',
                                 'Pursuing a Greater Purpose',
                                 'Social Status/Sociability',
                                 'Cultured/Well-traveled/"Woke"',
                                 'Self-Care/Personal Hygiene/Cleanliness',
                                 'Honesty/Dependable/Reliable',
                                 'Family Values'])

weights = [30,25,20,15,10]
j = 0
na = False
total_list=[]

for x in record:
    #print(x,len(x))
    ls = [0 for x in range(len(feat.columns))]
    
    i = 0
    for k in range(25,40,3):
        if None in x[25:40]:
            #print(x[25:40])
            na = True
            break
        if x[k] in feat.columns:
            ls[feat.columns.get_loc(x[k])] = weights[i]
        i += 1
    
    total = 0
    for k in range(65,95):
            #print(j)
            if x[k]:
                total += 1
    
    if na:
        na = False
        continue
    
    total_list.append(total*10)
    feat.loc[j,:] = ls
    j += 1
    
if conn:
    cur.close()
    conn.close()

print(len(total_list))
print(feat.shape)

model = create_model(feat,pd.DataFrame({0:total_list}),e=.4)

pickle.dump(model, open('NewFrontEnd/frontend/static/data/feat_model.sav', 'wb'))

69
(69, 17)
Training Score: 0.4074912535408274
Testing Score: 0.32314156108473935
Training Score: 0.377897323304062
Testing Score: 0.3157845102288367
Training Score: 0.3733699027889292
Testing Score: 0.4411534158184456


In [9]:
X = features
y = unsuccess_value


In [4]:
def bootstrap(X,y,target):
    X_ = X
    y_ = y
    while X_.shape[0] < target:
        n = np.random.randint(X.shape[0])
        X_ = X_.append(X.iloc[n,:])
        y_ = y_.append(y.iloc[n])
    return (X_,y_)

In [35]:
def create_model(X,y,num_samples=1000, e = .7):
    testing_score = 0
    while(testing_score < e):
        Xb,yb = bootstrap(X,y,num_samples)
        X_train, X_test, y_train, y_test = train_test_split(Xb, yb)
        model = LinearRegression()

#         # Fitting our model with all of our features in X
        model.fit(Xb, yb)
#         print(f"R2 Score: {model.score(Xb, yb)}")

        model.fit(X_train, y_train)
        testing_score = model.score(X_test, y_test)
        print(f"Training Score: {model.score(X_train, y_train)}")
        print(f"Testing Score: {testing_score}")
    return model

In [16]:
model = create_model(X,y)

Training Score: 0.714840499144723
Testing Score: 0.6945134157648472
Training Score: 0.7493452168471295
Testing Score: 0.7084411486791276


In [19]:
filename = 'NewFrontEnd/frontend/static/data/finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [22]:
new = model.predict(X)

In [23]:
print(new[0][0])

53.36018456619895


NameError: name 'unsuccess_value' is not defined